# Redes Neuronales

In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier, MLPRegressor

from sklearn.compose import make_column_selector as selector #Para seleccionar de forma automática las variables numéricas y categóricas
from sklearn.preprocessing import OneHotEncoder #Para codificar las variables categóricas usando dummies
from sklearn.preprocessing import StandardScaler #Para normalizar las variables numéricas
from sklearn.compose import ColumnTransformer #Modifica las columnas usando los preprocesadores
from sklearn.pipeline import make_pipeline #Planifica una secuencia de procesos
from sklearn import set_config #Para mostrar graficamente el pipeline
from sklearn.model_selection import GridSearchCV

set_config(display='diagram')
#Metrics
from sklearn.metrics import make_scorer, accuracy_score,precision_score, r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score, ConfusionMatrixDisplay

#### Lectura de Datos

In [27]:
# Leer los datos
df = pd.read_csv("train.csv")

# Selección de variables
variables_numericas = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
    '1stFlrSF', 'FullBath', 'YearBuilt', 'KitchenAbvGr',
    'TotRmsAbvGrd', 'Fireplaces', 'SalePrice'
]
df = df[variables_numericas]

In [28]:
df['SalePrice'] = np.log(df['SalePrice'])

In [29]:
X = df.drop(['SalePrice'], axis=1)
y = df['SalePrice']

In [30]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.3,train_size=0.7,random_state=0)

##### División en entrenamiento y prueba


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [32]:
scaler = StandardScaler()
X_train_r = scaler.fit_transform(X_train)
X_test_r = scaler.transform(X_test)

##### Creación de Modelos:

In [39]:
# === Modelo 1 Mejorado: ReLU + adam ===
modelo_reg1 = MLPRegressor(
    hidden_layer_sizes=(8, 4),
    activation='logistic',
    solver='lbfgs',  # más preciso en regresión pequeña
    learning_rate_init=0.001,
    max_iter=1000,
    early_stopping=True,
    random_state=42,
    verbose=True
)
modelo_reg1.fit(X_train_r, y_train)
y_pred_reg1 = modelo_reg1.predict(X_test_r)

# === Modelo 2 Mejorado: tanh + lbfgs ===
modelo_reg2 = MLPRegressor(
    hidden_layer_sizes=(6, 4),
    activation='tanh',
    solver='lbfgs',  # más preciso en regresión pequeña
    max_iter=1000,
    random_state=42
)
modelo_reg2.fit(X_train_r, y_train)
y_pred_reg2 = modelo_reg2.predict(X_test_r)

c:\Users\n8nug\Documents\Uni\2025 - 1er Semestre\Minería de Datos\PR2-MD\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\n8nug\Documents\Uni\2025 - 1er Semestre\Minería de Datos\PR2-MD\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


##### Métricas

In [41]:
def evaluar_modelo(y_true, y_pred, nombre):
    print(f"\n📊 Resultados - {nombre}")
    print(f"R² Score       : {r2_score(y_true, y_pred):.4f}")
    print(f"MSE (Error Cuadrático Medio) : {mean_squared_error(y_true, y_pred):.2f}")
    print(f"MAE (Error Absoluto Medio)   : {mean_absolute_error(y_true, y_pred):.2f}")

evaluar_modelo(y_test, y_pred_reg1, "Modelo logistic")
evaluar_modelo(y_test, y_pred_reg2, "Modelo tanh")




📊 Resultados - Modelo logistic
R² Score       : 0.8277
MSE (Error Cuadrático Medio) : 0.03
MAE (Error Absoluto Medio)   : 0.11

📊 Resultados - Modelo tanh
R² Score       : 0.8320
MSE (Error Cuadrático Medio) : 0.03
MAE (Error Absoluto Medio)   : 0.11
